In [ ]:
# Enable autoreload to automatically reload modules when they are updated
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
from typing import Dict, Any, Optional, Union
import joblib
import psutil
import os
import numpy as np
from sarpyx.processor.core.focus import CoarseRDA
from sarpyx.utils.zarr_utils import ZarrManager

# Only add path if not already added
cwd = Path.cwd()
sarpyx_path = cwd.parent
if str(sarpyx_path) not in __import__('sys').path:
    __import__('sys').path.append(str(sarpyx_path))
    
# Create output directory for stepped processing results
output_dir = cwd.parent / 'focused_data' 
output_dir.mkdir(parents=True, exist_ok=True)
print(f'📁 Output directory: {output_dir}')

In [ ]:
file_path = "/Data_large/marine/PythonProjects/SAR/sarpyx/decoded_data/s1a-s1-raw-s-hh-20150519t121652-20150519t121725-005990-007b7d_burst_0.zarr"
manager = ZarrManager(file_path)
raw_data = manager._export_raw()

def slice_section(data, k: int, N: int):
    """Slice data into the k-th section out of N sections."""
    total_length = len(data)
    section_size = total_length // N
    start_idx = k * section_size
    end_idx = (k + 1) * section_size if k < N - 1 else total_length
    return slice(start_idx, end_idx)

# Define the section parameters
k = 3  # Section index (0-based)
N = 4  # Total number of sections

# Apply slicing to raw_data
raw_data['echo'] = raw_data['echo'][slice_section(raw_data['echo'], k, N), :]
raw_data['metadata'] = raw_data['metadata'].iloc[slice_section(raw_data['metadata'], k, N)]

print(f'📊 Sliced raw data shape: {raw_data["echo"].shape}'
      f' | Metadata shape: {raw_data["metadata"].shape}')

In [ ]:
processor = CoarseRDA(
            raw_data=raw_data,
            verbose=False,
        )
print(f'🛠️ Processor initialized with raw data of shape: {raw_data["echo"].shape}')

In [ ]:
processor.data_focus()

In [ ]:
processor._display_slice(slice=(0,5000,0,35000), vmax=1200)